In [ ]:
pip install langchain_openai

In [1]:
import os

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

In [2]:
# from langchain_openai import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings(
#     base_url=os.environ['EMBEDDINGS_BASE_URL']
# )

In [2]:
from langchain.llms import Ollama
chat = Ollama(model="llama3.2:latest", temperature=0.1)

/var/folders/px/22qqx_qn399gggmw1gsvgk000000gn/T/ipykernel_46523/1596289383.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  chat = Ollama(model="llama3.2:latest", temperature=0.1)


In [3]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "請回答以下提問: \n{query}"
)

In [4]:
from langchain.schema.output_parser import StrOutputParser
output_parser = StrOutputParser()

In [5]:
from langchain.chains import LLMChain

chain = LLMChain(
    llm=chat,
    prompt=prompt_template,
    output_parser=output_parser
)

output = chain.invoke({"query": "生成式人工智能是什麼？"})
print(output["text"])

/var/folders/px/22qqx_qn399gggmw1gsvgk000000gn/T/ipykernel_46523/1852833169.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


答案：生成式人工智能（Generative AI）是一種人工智能技術，能夠創建新的內容或資料，例如圖像、音頻、文本等。它通過使用神經網路和其他算法來學習和模仿已有的資料，從而產生出類似原材料的新資料。


In [6]:
chain = prompt_template | chat | output_parser #LCEL

output = chain.invoke({"query:" "現在幾點？"})

print(output)

答案：我不知道現在的時間。


In [7]:
import datetime
from langchain.agents import tool

@tool
def check_time(format:str = "%Y-%m-%d %H:%M:%S"):
    """根據指定的格式返回當前日期和時間"""

    current_time = datetime.datetime.now()

    formatted_time = current_time.strftime(format)

    return formatted_time

In [8]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub
prompt_template = hub.pull("aixgeek/react4chinese")

tools = [check_time]

agent = create_react_agent(
    llm=chat,
    tools=tools,
    prompt=prompt_template
)

agent_executor = AgentExecutor(
    llm=chat,
    agent=agent,
    tools=tools,
    handle_parsing_errors=True,
    verbose=True
)

agent_executor.invoke({"input": "現在幾點？"})

/opt/homebrew/lib/python3.10/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
Action: check_time
Action Input: '%Y-%m-%d %H:%M:%S''2025-01-06 11:38:00'Question: 現在幾點？

Thought: Action: check_time
Action Input: '%Y-%m-%d %H:%M:%S'
'2025-01-06 11:38:00'
Question: 現在幾點？

Thought: Action: check_time
Action Input: '%Y-%m-%d %H:%M:%S''2025-01-06 11:38:01'Question: 現在幾點？
Thought: Action: check_time
Action Input: '%Y-%m-%d %H:%M:%S''2025-01-06 11:38:01'Question: 現在幾點？

Thought: Action: check_time
Action Input: '%Y-%m-%d %H:%M:%S'
'2025-01-06 11:38:01'
Question: 現在幾點？

Thought: Action: check_time
Action Input: '%Y-%m-%d %H:%M:%S''2025-01-06 11:38:02'Question: 現在幾點？
Thought: Action: check_time
Action Input: '%Y-%m-%d %H:%M:%S''2025-01-06 11:38:02'Question: 現在幾點？

Thought: Action: check_time
Action Input: '%Y-%m-%d %H:%M:%S'
'2025-01-06 11:38:03'
Question: 現在幾點？
Thought: Action: check_time
Action Input: '%Y-%m-%d %H:%M:%S''2025-01-06 11:38:03'Question: 現在幾點？

Thought: Action: check_time
Action Input: '%Y-%m-%d %H:%M:%S'
'2025-01-06 1

{'input': '現在幾點？',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [9]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.agents import create_react_agent,AgentExecutor
from langchain import hub
prompt = hub.pull("aixgeek/react4chinese")

/opt/homebrew/lib/python3.10/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
pip install wikipedia

In [12]:
api_wrapper = WikipediaAPIWrapper(
    top_k_results=1,
    doc_content_chars_max=500
)
wiki_tool=WikipediaQueryRun(
    api_wrapper=api_wrapper
)
tools = [wiki_tool]

In [13]:
agent = create_react_agent(
    llm = chat,
    tools = tools,
    prompt=prompt
)
agent_executor = AgentExecutor(
    agent=agent,
    handle_parsing_errors=True,
    verbose=True,
    tools=tools
)

In [14]:
input = "目前台灣總統是誰？"
agent_executor.invoke({"input":input})



> Entering new AgentExecutor chain...
Question: 目前台灣總統是誰？

Thought: 我們需要查找目前台湾的总统。

Action: wikipedia
Action Input: "台湾总统"Page: President of the Republic of China
Summary: The president of the Republic of China, also known as the president of Taiwan, is the head of state of the Republic of China, commonly known as Taiwan, as well as the commander-in-chief of the Republic of China Armed Forces. Before 1949 the position had the authority of ruling over Mainland China, but after communist victory in the Chinese Civil War, the remaining jurisdictions of the ROC have been limited to Taiwan, Penghu, Kinmen, Matsu, and smQuestion: 目前台灣總統是誰？

Thought: 我們需要查找目前台湾的总统。

Action: wikipedia
Action Input: "台湾总统"Page: President of the Republic of China
Summary: The president of the Republic of China, also known as the president of Taiwan, is the head of state of the Republic of China, commonly known as Taiwan, as well as the commander-in-chief of the Republic of China Armed Forces. Before 1949 the p

{'input': '目前台灣總統是誰？', 'output': '蔡英文'}

In [15]:
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(

    openai_api_key=os.environ["OPENAI_API_KEY"],
    temperature=0.1
)

In [18]:
from crewai import Agent,Task,Crew,Process

In [21]:
#第一個AI員工
researcher = Agent(
    role = "市場搜尋官",
    goal='搜索關於美食訊息',
    backstory='你是一個經驗豐富的市場調研助理',
    allow_delegation=True,
    verbose=True
)
analyst = Agent(
    role = "市場分析員",
    goal='分析市面哪些美食最受歡迎',
    backstory='你是一個專業市場分析員',
    allow_delegation=False,
    verbose=True
)
task1 = Task(
    description = "分析最新市場上最受歡迎的美食",
    agent = researcher
)
task2 = Task(
    description = "寫一份完美的市場分析，分析哪些美食最受歡迎.",
    agent = analyst
)

crew = Crew(
    agents=[researcher,analyst],
    task=[task1,task2],
    process=Process.sequential,
    verbose=2
)
result = crew.kickoff()

ValidationError: 1 validation error for Task
expected_output
  Field required [type=missing, input_value={'description': '分析...的市場調研助理)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing